## Ipython Notebooks Templates

Add the package path for testing

In [1]:
import sys
sys.path.append('E:\\git_folder\\quant_models')

## Start Testing

In [2]:
from util.metrics import sharpe_iid, compute_deflated_sharpe_ratio, \
        sharpe_non_iid, adjusted_sharpe, sharpe_iid_adjusted          

#### Part 1 Simulate Return Matrix

The logics is given as:

1. Generate N random algos that each list consists of 1, 0, -1
2. Apply the N algos on the XU1 price to get the return 
3. Resample the PNL to daily PNL

In [3]:
import pandas as pd
from util.utils import sample_df, sample_pnl
import numpy as np
import random
random.seed(30)
np.random.seed(20)

In [4]:
import pandas as pd
data_fn = "././data/xu1_1min.csv"
df_price = pd.read_csv(data_fn)

In [5]:
test_df = df_price.copy()
test_df.loc[:,'datetime'] = pd.to_datetime(test_df.datetime)
test_df.set_index('datetime', inplace=True)
test_df = test_df.loc[test_df.index > pd.to_datetime("2017-01-01"),:]

In [6]:
df_5min = sample_df(test_df, "30T")

In [7]:
df_5min['pct'] = df_5min.open.pct_change()
df_5min.dropna(inplace=True)

In [8]:
N_data = df_5min.count()['open']

In [9]:
# random generate strategy
N_stat = 1000
bs_stats = [np.random.randint(3, size=N_data) -1 for idx in range(N_stat)]
bs_stats = np.array(bs_stats).transpose()

In [10]:
print(bs_stats.shape)

(31444, 1000)


In [11]:
df_5min = df_5min[['pct']]

In [12]:
df_5min.shape

(31444, 1)

In [13]:
for idx in range(N_stat):
    # trade signl * pct will be the pnl
    col_name = 'strat_{}'.format(idx)
    df_5min[col_name] = bs_stats[:,idx]
    df_5min[col_name] = df_5min['pct'] * df_5min[col_name]

### 

In [14]:
df_5min.drop(columns=['pct'], inplace=True)

In [15]:
df_5min.head()

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_990,strat_991,strat_992,strat_993,strat_994,strat_995,strat_996,strat_997,strat_998,strat_999
datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:00:00,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000
2017-01-03 09:30:00,-0.001754,0.000000,-0.001754,0.001754,0.001754,-0.001754,0.000000,0.001754,-0.001754,-0.001754,...,-0.001754,-0.001754,0.000000,0.001754,0.000000,0.001754,0.001754,0.001754,-0.001754,0.000000
2017-01-03 10:00:00,0.012253,0.000000,-0.012253,0.000000,0.000000,-0.012253,0.012253,0.000000,0.000000,0.012253,...,-0.012253,0.012253,0.012253,0.012253,-0.012253,0.012253,-0.012253,-0.012253,0.000000,-0.012253
2017-01-03 10:30:00,0.000000,-0.002470,0.000000,-0.002470,0.002470,0.002470,-0.002470,-0.002470,-0.002470,0.002470,...,0.002470,-0.002470,0.000000,-0.002470,0.002470,-0.002470,0.002470,0.002470,0.002470,0.002470
2017-01-03 11:00:00,0.004189,-0.004189,0.004189,-0.004189,-0.004189,-0.004189,-0.004189,-0.004189,-0.000000,-0.004189,...,-0.004189,-0.004189,-0.004189,-0.000000,-0.004189,0.004189,-0.000000,-0.000000,0.004189,-0.000000


In [16]:
pnl_daily = sample_pnl(df_5min,'1D', cum_mode=False)

In [17]:
pnl_daily.head()

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_990,strat_991,strat_992,strat_993,strat_994,strat_995,strat_996,strat_997,strat_998,strat_999
datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-03,0.009494,-0.005924,-1.278737e-02,-0.006145,-0.003174,-0.020175,0.001387,-0.005148,-0.007446,0.018434,...,-1.548603e-02,0.009034,0.007315,0.013514,-0.005059,0.018204,-0.012235,-0.010004,0.002186,-0.012251
2017-01-04,0.000474,-0.004704,-5.443511e-03,0.002462,-0.002726,-0.001481,0.007389,-0.002215,-0.004670,0.001990,...,-4.211409e-03,-0.004445,-0.002232,0.002981,-0.002209,-0.001024,-0.001735,0.006659,-0.003709,0.007663
2017-01-05,-0.000492,-0.000495,-8.588066e-03,0.005393,0.005150,0.003437,0.000991,-0.005887,0.002461,0.000728,...,-2.940379e-03,0.003179,0.004421,-0.006873,-0.010799,-0.000475,0.003431,-0.005897,-0.000249,-0.004913
2017-01-06,-0.002213,-0.001721,2.214737e-03,-0.001721,-0.000974,-0.003691,-0.004182,0.004674,-0.002216,0.010088,...,4.929711e-03,0.003199,-0.001472,-0.001978,-0.000980,-0.001967,0.004427,-0.004164,0.003942,0.005168
2017-01-07,-0.000988,0.000247,-6.095127e-08,-0.000247,-0.001235,0.000494,-0.000247,0.000494,0.000741,-0.000494,...,3.047865e-07,-0.000247,-0.000741,0.000247,0.000247,-0.000741,-0.000988,-0.001235,0.001235,-0.001235


In [18]:
# Split Pnl_daily into insample and outsample
pnl_insample  = pnl_daily.loc[pnl_daily.index <= pd.to_datetime('2017-06-01'), :]
pnl_outsample = pnl_daily.loc[pnl_daily.index >  pd.to_datetime('2017-06-01'), :]

#### Part 2 Testing Sharpe ratio and its variants

* IID Sharpe
* IID Sharpe Adjusted
* Sharpe_non_iid

In [19]:
rtns_pct = pnl_daily.strat_105.values
print(type(rtns_pct))
print(rtns_pct.shape)

<class 'numpy.ndarray'>
(1080,)


In [20]:
# normal sharpe
sharpe = sharpe_iid(rtns_pct, bench=0, factor=252, log=False)
print(sharpe)

0.37415457695908105


In [21]:
# IID adjusted sharpe
adj_sharpe = sharpe_iid_adjusted(rtns_pct, bench=0, factor=252, log=False)
print(adj_sharpe)

0.3738352098953136


In [22]:
# None IID adjusted sharpe
nonIIDsharpe = sharpe_non_iid(rtns_pct, bench=0, factor=252, p_critical=0.05, log=False)
# since the algo is random, the Ljung-Box test for autocorrelation can be passed
# the value is the same as the normal sharpe
print(nonIIDsharpe)

0.6057909173116635


#### Part 3 Compute the deflated ration for the shapre

* it will consider the insample and outsample performance
* 
* from in_sample: the number of trials, the std of computed sharpe   
  -> reduce the selection bias
* the return sequence for estimated SR: the number of days,  the skew and kurtosis of the return sequence, the sharpe  
  -> reduce the non-normal bias
* The computed deflated sharpe is in the range (0, 1). The less, the better

In [23]:

sharpe_list = pnl_insample.apply(lambda x: sharpe_iid(x, factor=252, log=False))

In [24]:
sharpe_list.index

Index(['strat_0', 'strat_1', 'strat_2', 'strat_3', 'strat_4', 'strat_5',
       'strat_6', 'strat_7', 'strat_8', 'strat_9',
       ...
       'strat_990', 'strat_991', 'strat_992', 'strat_993', 'strat_994',
       'strat_995', 'strat_996', 'strat_997', 'strat_998', 'strat_999'],
      dtype='object', length=1000)

###  The best algo in the insample period

In [25]:
max_sharpe = sharpe_list.max()
std_sharpe = sharpe_list.std()
bset_strat = sharpe_list.idxmax()
nb_trials  = N_stat

In [26]:
print('best random algo is {} with a sharpe {}'.format(bset_strat, max_sharpe))

best random algo is strat_150 with a sharpe 4.059115028336026


####  The selected algo in the outsample period

In [27]:
test_pnl_oos = pnl_outsample[bset_strat]
test_rtns = test_pnl_oos.values
test_pnl_is = pnl_insample[bset_strat]
test_rtns_is = test_pnl_is.values
estimated_sharpe = sharpe_iid(test_rtns, factor=252, log=False)

#### Compute the deflation of In-sample SR Estimation

In [28]:
print('sharpe computed: {}'.format(max_sharpe))

sharpe computed: 4.059115028336026


In [29]:
ratio_deflated = compute_deflated_sharpe_ratio(max_sharpe, std_sharpe, nb_trials, test_rtns_is)

In [30]:
print(ratio_deflated)

0.5450484738522648


#### Compute the deflation of OOS SR Estimation

In [31]:
print('sharpe computed: {}'.format(estimated_sharpe))

sharpe computed: -0.3289545643424261


In [32]:
ratio_deflated = compute_deflated_sharpe_ratio(estimated_sharpe, std_sharpe, nb_trials, test_rtns)

In [33]:
print(ratio_deflated)

0.0


###  Pass the test

Since the best strat is totally data mining producted (randomly selected), it should not be used even if the in-sample sharpe is almost 4. Then, the computed deflated ratio is 0, which can filter the strategy as a false discovery. The ratio can be understood as the probability of the tested algo may have a postive sharpe in the future. The higher, the better